# DevMeeting – Data Science
## 3. Keras + TensorFlow

![tensorflow](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/Tensorflow_logo.svg/100px-Tensorflow_logo.svg.png)

TensorFlow to biblioteka rozwijana przez Google, służąca do implementacji algorytmów głębokiego uczenia, głównie - głębokich sieci neuronowych. Nakładką na TensorFlow jest tzw. __Keras__, który udostępnia wysokopoziomowe API do pracy z metodami głębokiego uczenia.

Metoda opisywana w tym notebooku wymaga, do szybkiego działania, dostępności karty graficznej, na potrzeby obliczeń. Aby skorzystać z darmowego dostępu w ramach Colaboratory, należy wybrać opcję akceleracji sprzętowej w opcjach __Runtime__->__Change runtime type__->__Hardware accelerator__->__GPU__.

In [0]:
import numpy as np
import pandas as pd

from PIL import Image

## 3.1 Podstawy Głębokiego Uczenia

## 3.2 Model predykcji wieku i płci

W tej części skorzystamy z gotowej implementacji modelu do predykcji wieku i płci ze zdjęć. W pierwszej kolejności musimy pobrać kod definiujący architekturę sieci neuronowej:

In [0]:
!git clone https://github.com/yu4u/age-gender-estimation

Musimy również pobrać sam zapisany model, czyli liczby opisujące połączenia pomiędzy kolejnymi neuronami w głębokiej sieci neuronowej (tzw. **wagi**).

In [0]:
!wget https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/weights.29-3.76_utk.hdf5

Skorzystamy z modelu jak z modułu w Pythonie - dodajemy pobrane repozytorium do zmiennej środowiskowej PATH, a następnie importujemy klasę `WideResNet`, która stanowi definicję architektury sieci neuronowej.

In [0]:
# potrzebne, by zaimportować pobrane repozytorium
import sys
sys.path.append("./age-gender-estimation")

from wide_resnet import WideResNet

Budujemy model: tworzymy obiekt klasy `WideResNet`, a następnie wywołujemy go jak funkcję - skutkuje to zbudowaniem właściwego grafu obliczeniowego.

In [0]:
deep_model = WideResNet(image_size=64)
deep_model = deep_model()

W tym momencie model ma poprawną architekturę, ale brakuje jeszcze wag, które określają jego działanie. Kolejnym krokiem będzie zatem wczytanie zapisanych wag do utworzonej architektury.

In [0]:
deep_model.load_weights("/content/weights.29-3.76_utk.hdf5")
deep_model.summary()

Powyższa lista opisuje architekturę sieci neuronowej - istotne są dwie rzeczy:
 * warstwa wejściowa - `input_4 (InputLayer)`
 * warstwy wyjściowe - `pred_gender` i `pred_age`
 
Do każdej z tych warstw przypisany jest jej wymiar, czyli kształt wektora lub macierzy, jaki zostanie zwrócony.
W przypadku warstwy wejściowej jest to `(None, 64, 64, 3)`, co oznacza, że warstwa ta przyjmuje dowolną liczbę (element `None`) obrazów o wymiarach 64x64, każdy z trzema kanałami (RGB).

Predykcja, przy użyciu modelu, odbywa się poprzez zastosowanie metody `predict(input)`. 
Model zwraca parę wektorów: `(predykcja_plci, predykcja_wieku)`

Konwolucyjne sieci neuronowe, implementowane w TensorFlow, wymagają szczególnego formatu wejścia - n-wymiarowej tablicy, wg poniższego wzoru:
```
(liczba_przypadków, szerokość, wysokość, głębia)
```

* liczba przypadków określa ile na raz obrazów przetwarzanych jest przez sieć
* szerokość, wysokość określają wymiary każdego z obrazów
* głębia określa liczbę kanałów obrazu - 3 dla obrazów RGB, 1 dla obrazów w skali szarości.

Taki format wygodny jest przy procesie uczenia sieci, ale przy przetwarzaniu pojedynczych przykładów `liczba_przypadków` zwykle równa będzie 1. Zakładając, że wczytany obraz reprezentowany będzie przez tablicę o wymiarach `(szerokość, wysokość, głębia)`, konieczne jest sztuczne dodanie jednego wymiaru na początku, by uzyskać format `(1, szerokość, wysokość, głębia)`.

Operację tę można wykonać przy użyciu NumPy, korzystając z metody `.reshape(1, szerokość, wysokość, głębia)`.

In [0]:
random_input = np.random.random((1, 64, 64, 3))

prediction = deep_model.predict(random_input)
print(prediction)
gender, age = prediction

print(gender.shape)
print(age.shape)

In [0]:
import matplotlib.pyplot as plt

age_values = list(range(len(age[0])))
print(age_values)
plt.bar(age_values, age[0])

Model zwraca predykcję w formie pary tablic NumPy:
```python
(gender, age)
```

Tablice te określają rozkłady prawdopodobieństw wystąpienia danej płci lub danego wieku. Mając takie rozkłady, potrzebujemy wybrać z nich najbardziej prawdopodobną wartość.

## 3.3 Praca z obrazami w Pythonie

In [0]:
from PIL import Image

Do wczytywania i obsługi obrazów wykorzystami bibliotekę PIL. Zawiera ona klasę `Image`, która reprezentuje obrazek. Dla przykładu pobierzemy popularny obrazek testowy - [Lenna](https://en.wikipedia.org/wiki/Lenna).

In [0]:
!wget -O lenna.jpg https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png

In [0]:
lenna = Image.open("lenna.jpg")
lenna

Klasa `Image` implementuje wiele przydatnych metod do pracy z obrazami, m.in: `resize((width, height))`, która skaluje obraz do zadanej wielkości.

In [0]:
lenna = lenna.resize((100, 100))
lenna

Aby możliwe było przekazanie obrazu do konwolucyjnej sieci neuronowej, wymagane jest przekształcenie go do postaci tablicy NumPy

In [0]:
np_lenna = np.array(lenna)
np_lenna.shape

Należy pamiętać o konieczności zmiany kształtu tablicy reprezentującej obraz, by uwzględnić dodatkowy pierwszy wymiar.

# Zadania do wykonania

1. Przygotuj potok przetwarzania obrazów:
  * wczytanie obrazu z pliku (`Image.open()`)
  * przeskalowanie obrazu do rozmiaru akceptowanego przez sieć neuronową
  * przekonwertowanie do postaci tablicy NumPy (z uwzględnieniem wymiaru batcha)
  * przetworzenie przez model
  * pozyskanie wyników (`np.argmax`)
  * zwrócenie wyniku przetwarzania w formacie JSON/słownika, np:
  
```python
      {
        "gender": "female",
        "age": 30
      }
```


In [0]:
age, gender = None, None

# miejsce na kod


{
  "gender": gender,
  "age": age
}